<a href="https://colab.research.google.com/github/sks01dev/HF/blob/main/Preprocessing_datasets_GLUE_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers[sentencepiece] evaluate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00


1. SST2

In [5]:
raw_data

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [7]:
raw_data['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [9]:
import pandas as pd

df = pd.DataFrame(raw_data['train'])

In [10]:
df[:5]

,sentence,label,idx
0,hide new secretions from the parental units,0,0
1,"contains no wit , only labored gags",0,1
2,that loves its characters and communicates som...,1,2
3,remains utterly satisfied to remain the same t...,0,3
4,on the worst revenge-of-the-nerds clichés the ...,0,4


In [11]:
raw_data['train'].features

{'sentence': Value('string'),
 'label': ClassLabel(names=['negative', 'positive']),
 'idx': Value('int32')}

In [12]:
# 0-> negative
# 1-> positive

In [44]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

def glue_preprocessing_sst2(task, checkpoint):
    raw_data = load_dataset('glue', task)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

    def tokenize(input):
        return tokenizer(input['sentence'], truncation=True)

    # tokenize the dataset
    tokenized_data = raw_data.map(tokenize, batched=True)

    # drop unused columns
    tokenized_data = tokenized_data.remove_columns(['sentence', 'idx'])

    # for batch processing
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    return tokenized_data, data_collator

In [45]:
tokenized_data_sst, data_collator = glue_preprocessing_sst2('sst2', 'bert-base-uncased')

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [39]:
tokenized_data_sst['train'].features

{'label': ClassLabel(names=['negative', 'positive']),
 'input_ids': List(Value('int32')),
 'token_type_ids': List(Value('int8')),
 'attention_mask': List(Value('int8'))}

In [41]:
# create batch
batch = data_collator(list(tokenized_data_sst['train']))

In [43]:
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([67349, 66]),
 'token_type_ids': torch.Size([67349, 66]),
 'attention_mask': torch.Size([67349, 66]),
 'labels': torch.Size([67349])}

2. MRPC

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

def glue_preprocessing_mrpc(checkpoint):
    raw_data = load_dataset('glue', 'mrpc')
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def tokenize(input):
        return tokenizer(input['sentence1'], input['sentence2'], truncation=True)

    # tokenize the dataset
    tokenized_data = raw_data.map(tokenize, batched=True)

    # drop unused columns
    tokenized_data = tokenized_data.remove_columns(['sentence1', 'sentence2', 'idx'])

    # for batch processing
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    return tokenized_data, data_collator

In [2]:
tokenized_data_mrpc, data_collator = glue_preprocessing_mrpc('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
tokenized_data_mrpc['train'].features

{'label': ClassLabel(names=['not_equivalent', 'equivalent']),
 'input_ids': List(Value('int32')),
 'token_type_ids': List(Value('int8')),
 'attention_mask': List(Value('int8'))}